<a href="https://colab.research.google.com/github/LucasGleysson/Estatistica_DS_ML/blob/master/Exercicio_Amostragem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercicio de comparação das amostragens

##Visualização e tratamento dos dados

In [39]:
import pandas as pd
import numpy as np
import random

data = pd.read_csv('/content/drive/MyDrive/Datasets/credit_data.csv')
data.head()

,i#clientid,income,age,loan,c#default
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0
3,4,42709.534201,45.751972,6103.642260,0
4,5,66952.688845,18.584336,8770.099235,1


In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   i#clientid  2000 non-null   int64  
 1   income      2000 non-null   float64
 2   age         1997 non-null   float64
 3   loan        2000 non-null   float64
 4   c#default   2000 non-null   int64  
dtypes: float64(3), int64(2)
memory usage: 78.2 KB


In [41]:
# Corrigindo valores NA
data['age'] = data['age'].fillna(data['age'].mean())

# Convertendo o tipo
data['age'] = data['age'].astype(int)

# Corrigindo valores errados
data['age'] = data['age'].replace([-28, -52, -36], [28, 52, 36])

# Arredondando valores
data['income'] = data['income'].round(2)
data['loan'] = data['loan'].round(2)

In [42]:
data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
i#clientid,2000.0,1000.500000,577.494589,1.00,500.7500,1000.50,1500.2500,2000.00
income,2000.0,45331.599930,14326.327142,20014.49,32796.4575,45789.12,57791.2850,69995.69
age,2000.0,40.425500,13.262594,18.00,29.0000,41.00,52.0000,63.00
loan,2000.0,4444.369635,3045.409995,1.38,1939.7050,3974.72,6432.4075,13766.05
c#default,2000.0,0.141500,0.348624,0.00,0.0000,0.00,0.0000,1.00


## Funções de amostragens

In [43]:
# Amostragem aleatória simples

def amostragem_aleatoria_simples(dataset, qnt_amostra):
  amostra = dataset.sample(qnt_amostra)
  return amostra

In [44]:
# Amostragem sistemática

def amostragem_sistematica(dataset, qnt_amostra):
  intervalo = len(dataset) / qnt_amostra
  inicio = random.randint(0, intervalo)
  sequencia = np.arange(inicio, len(dataset), intervalo)

  amostra = data.iloc[sequencia]
  return amostra

In [45]:
# Amostragem por grupos

def amostragem_por_grupos(dataset, qnt_grupos):
  dataset = dataset.copy()
  qnt_amostra = len(dataset) / qnt_grupos

  grupo = []
  id_grupo = 1
  contador = 0
  for _ in range(len(dataset)):
    grupo.append(id_grupo)
    contador += 1
    if contador == qnt_amostra:
      id_grupo += 1
      contador = 0
  dataset['grupo'] = grupo
  
  gp_selecionado = random.randint(1, qnt_grupos)
  amostra = dataset.loc[dataset['grupo'] == gp_selecionado]
  return amostra

In [46]:
# Amostragem Estratificada

def amostragem_estratificada(dataset, qnt_amostra, gp_divisor):
  from sklearn.model_selection import StratifiedShuffleSplit

  qnt_amostra = qnt_amostra / len(dataset)
  split = StratifiedShuffleSplit(test_size = qnt_amostra)
  for _, y in split.split(dataset, dataset[gp_divisor]):
    df_y = dataset.iloc[y]

  amostra = df_y
  return amostra

In [47]:
# Amostragem de reservatório

def amostragem_reservatorio(dataset, qnt_amostra):
  stream = []
  for i in range(len(dataset)):
    stream.append(i)
  
  reservatorio = [0] * qnt_amostra
  for i in range(qnt_amostra):
    reservatorio[i] = stream[i]

  i = 0
  tamanho = len(dataset)
  while i < tamanho:
    j = random.randrange(i + 1)
    if j < qnt_amostra:
       reservatorio[j] = stream[i]
    i += 1

  amostra = dataset.iloc[reservatorio]
  return amostra

## Aplicação das funções

In [52]:
estratificada = amostragem_estratificada(data, 1000, 'c#default')
simples = amostragem_aleatoria_simples(data, 1000)
reservatorio = amostragem_reservatorio(data, 1000)
sistematica = amostragem_sistematica(data, 1000)
grupos = amostragem_por_grupos(data, 4)

## Comparação dos resultados

In [78]:
comparacao = [['Dataset', data['age'].mean(), data['age'].median()],
              ['Simples', simples['age'].mean(), simples['age'].median()],
              ['Sistematica', sistematica['age'].mean(), sistematica['age'].median()],
              ['Estratificada', estratificada['age'].mean(), estratificada['age'].median()],
              ['Grupos', grupos['age'].mean(), grupos['age'].median()],
              ['Reservatorio', reservatorio['age'].mean(), reservatorio['age'].median()]]

comparacao = pd.DataFrame(comparacao, columns=['Amostragem', 'Media','Mediana'])
comparacao['Media'] = comparacao['Media'].astype(int)
comparacao['Mediana'] = comparacao['Mediana'].astype(int)
comparacao.sort_values(by='Media',ascending=False)

,Amostragem,Media,Mediana
4,Grupos,41,43
0,Dataset,40,41
2,Sistematica,40,41
1,Simples,39,39
3,Estratificada,39,40
5,Reservatorio,39,40
